# FanGraphs Data Scrape

*Gathering FanGraphs data for use in monitoring player performance.*

## Setup

In [ ]:
import os
import time
import pandas as pd
import sqlite3
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm_notebook

### Prepare Database

CSV files are downloaded to the specified directory and inserted into tables in a SQLite database. Existing files should be deleted and if the database does not exist, it will be creating upon connect.

Tables are not specified before creation simply to save time. Should a field change or be added, a complete reload is recommended.

In [ ]:
csvpath = "./FanGraphs Leaderboard.csv"

try:
    os.remove(csvpath)
except OSError:
    pass

fgdb = sqlite3.connect('./FanGraphs.db')

first_year = 2016
latest_year = 2017

### Specify WebDriver

This implementation is for OSX. It can be downloaded from [Google](https://sites.google.com/a/chromium.org/chromedriver/downloads) directly and unzipped to the repositories main directory.

In [ ]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory": os.getcwd()}
chromeOptions.add_experimental_option("prefs", prefs)
chromedriver = os.getcwd() + "/chromedriver"

cap = DesiredCapabilities.CHROME
cap["pageLoadStrategy"] = "none"

### Function to Replace Characters in Field Names

SQLite databases are less flexible than Pandas dataframes in terms of acceptable field names. This function accounts for these situations and adds variables as necessary. 

In [ ]:
repdict = {' ': '_',
           '%': 'pct',
           '(': '',
           ')': '',
           '.': '',
           '-': 'm',
           '/': 'd',
           '+': 'p',
           '1B': 'H1B',
           '2B': 'H2B',
           '3B': 'H3B'}


def clean_file(filepath, level, league='', season='', position=''):
    """Clean data prior to loading to database"""
    leaderboard = pd.read_csv(filepath)
    leaderboard['Level'] = level
    if season != '':
        leaderboard['Season'] = season
    else:
        pass
    if league != '':
        leaderboard['League'] = league
    else:
        pass
    if position != '':
        leaderboard['Position'] = position
    else:
        pass
    cols = list(leaderboard)
    for i in range(len(cols)):
        for key in repdict:
            cols[i] = cols[i].replace(key, repdict[key])
    leaderboard.columns = cols
    return leaderboard

## MLB Batting Data

Non-pitch data from 1990 to 2017 (season to-date). Details can be found directly in the report [link](http://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=c,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,102,103,104,105,106,107,108,109,110,111,198,199,200,201,203,204,205,206,207,208,209,210,211,212&season=2017&month=0&season1=2017&ind=0&team=&rost=&age=&filter=&players=
). Position, height, and weight are not included. 

In [ ]:
for year in tqdm_notebook(range(first_year, latest_year+1), desc='Years'):

    link = "http://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=c,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,102,103,104,105,106,107,108,109,110,111,198,199,200,201,203,204,205,206,207,208,209,210,211,212&season={}&month=0&season1={}&ind=0&team=&rost=&age=&filter=&players=".format(str(year), str(year)) # noqa
    driver = webdriver.Chrome(executable_path=chromedriver,
                              chrome_options=chromeOptions,
                              desired_capabilities=cap)
    wait = WebDriverWait(driver, 20)
    driver.get(link)
    wait.until(EC.presence_of_element_located((By.ID, "LeaderBoard1_cmdCSV")))
    driver.execute_script("window.stop();")

    # Click the Export Data button
    driver.execute_script("window.scrollTo(0, 200)")
    driver.find_element_by_id("LeaderBoard1_cmdCSV").click()

    # Wait until file downloads successfully
    while not os.path.exists(csvpath):
        time.sleep(0.1)
    if os.path.isfile(csvpath):

        # Import data and insert into table
        fglb = clean_file(csvpath, level="MLB", season=year)
        pd.io.sql.to_sql(fglb, name='majorsb', con=fgdb, if_exists='append')

        try:
            os.remove(csvpath)
        except OSError:
            pass
        time.sleep(2)
    else:
        raise ValueError("{} does not exist".format(csvpath))
    driver.close()

# Delete duplicates
fgdb.execute("""DELETE
                FROM majorsb
                WHERE rowid NOT IN (SELECT MIN(rowid)
                FROM majorsb
                GROUP BY Season, Level, Team, Name, playerid)""")

## MLB Pitching Data

Non-pitch data from 1990 to 2017 (season to-date). Details can be found directly in the report [link](http://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=c,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,211,212,213,214,215,217,218,219,220,221,222,223,224,225&season=2017&month=0&season1=2017&ind=0&team=&rost=&age=&filter=&players=). Height and weight are not included. 

In [ ]:
for year in tqdm_notebook(range(first_year, latest_year+1), desc='Years'):

    link = "http://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=c,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,211,212,213,214,215,217,218,219,220,221,222,223,224,225&season={}&month=0&season1={}&ind=0&team=&rost=&age=&filter=&players=".format(str(year), str(year)) # noqa
    driver = webdriver.Chrome(executable_path=chromedriver,
                              chrome_options=chromeOptions,
                              desired_capabilities=cap)
    wait = WebDriverWait(driver, 20)
    driver.get(link)
    wait.until(EC.presence_of_element_located((By.ID, "LeaderBoard1_cmdCSV")))
    driver.execute_script("window.stop();")

    # Click the Export Data button
    driver.execute_script("window.scrollTo(0, 200)")
    driver.find_element_by_id("LeaderBoard1_cmdCSV").click()

    # Wait until file downloads successfully
    while not os.path.exists(csvpath):
        time.sleep(0.1)
    if os.path.isfile(csvpath):

        # Import data and insert into table
        fglb = clean_file(csvpath, level="MLB", season=year)
        pd.io.sql.to_sql(fglb, name='majorsp', con=fgdb, if_exists='append')

        try:
            os.remove(csvpath)
        except OSError:
            pass
        time.sleep(2)
    else:
        raise ValueError("{} does not exist".format(csvpath))
    driver.close()

# Delete duplicates
fgdb.execute("""DELETE
                FROM majorsp
                WHERE rowid NOT IN (SELECT MIN(rowid)
                FROM majorsp
                GROUP BY Season, Level, Team, Name, playerid)""")

In [ ]:
fgdb.commit() 
fgdb.close()